In [1]:
# Установка необходимых версий библиотек:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r "/content/drive/MyDrive/GO2025/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Блоки кода ниже запускаются в первую очередь и один раз

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "Vikhrmodels/QVikhr-2.5-1.5B-Instruct-r"
# save_path = "qvikhr_model"

# # Скачиваем и сохраняем модель и токенизатор
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

# tokenizer.save_pretrained(save_path)
# model.save_pretrained(save_path)


In [5]:
# !cp -r qvikhr_model /content/drive/MyDrive/

# Выполните блоки кода ниже, чтобы развернуть веб-сервис

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# Копируем модель из Google Drive в текущую сессию
!cp -r /content/drive/MyDrive/qvikhr_model /content/qvikhr_model

# грузит примерно 3 минуты

In [7]:
# # Устанавливаем зависимости
# !pip install -q streamlit pyngrok transformers sentence-transformers
# !pip install --upgrade sentence-transformers huggingface-hub
# !pip install --upgrade streamlit
# !pip install fpdf2
# !apt-get install -y fonts-dejavu

# # грузит примерно 3 минуты

In [8]:
!apt-get install -y fonts-dejavu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra
The following NEW packages will be installed:
  fonts-dejavu fonts-dejavu-core fonts-dejavu-extra
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 3,085 kB of archives.
After this operation, 10.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-dejavu all 2.37-2build1 [3,192 B]
Fetched 3,085 kB in 1s (2,845 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpack

In [13]:
# Убиваем старые процессы, если сессия была запущена ранее
import os
os.system("pkill streamlit")
os.system("pkill ngrok")

# Настройка ngrok
!ngrok authtoken 2wX3kWVgB7whSHgoOeHwUCVhGvJ_7664QBav1Da6yXVfiLaRn

# Проброс порта
from pyngrok import ngrok
public_url = ngrok.connect(8501)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
%%writefile app.py
import streamlit as st
import time
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util
import base64
import os
from fpdf import FPDF
from fpdf.fonts import FontFace
import urllib.request

@st.cache_resource
def load_model():
    local_path = "/content/qvikhr_model"
    tokenizer = AutoTokenizer.from_pretrained(local_path)
    model = AutoModelForCausalLM.from_pretrained(local_path)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe, tokenizer

@st.cache_resource
def load_metric_model():
    return SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def count_syllables_ru(word):
    return len(re.findall(r'[аеёиоуыэюя]', word.lower()))

def flesch_readability_russian(text):
    sentences = re.split(r'[.!?]+', text)
    sentences = [s for s in sentences if s.strip()]
    words = re.findall(r'\b\w+\b', text)
    syllables = sum(count_syllables_ru(word) for word in words)

    num_sentences = len(sentences)
    num_words = len(words)

    if num_sentences == 0 or num_words == 0:
        return 0

    score = 206.835 - 1.3 * (num_words / num_sentences) - 60.1 * (syllables / num_words)
    return round(score, 2)

def generate_pdf(history):
    pdf = FPDF()
    pdf.add_page()

    # Скачиваем шрифт, если не установлен
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    font_name = "DejaVu"

    if not os.path.exists(font_path):
        os.makedirs(os.path.dirname(font_path), exist_ok=True)
        print("⏬ Скачиваем DejaVuSans.ttf...")
        url = "https://github.com/dejavu-fonts/dejavu-fonts/raw/version_2_37/ttf/DejaVuSans.ttf"
        urllib.request.urlretrieve(url, font_path)

    # Подключаем шрифт с поддержкой Unicode
    pdf.add_font(font_name, '', font_path, uni=True)
    pdf.set_font(font_name, size=12)

    pdf.cell(0, 10, txt="История упрощения определений", ln=True, align='C')
    pdf.ln(10)

    for i, item in enumerate(history, 1):
        pdf.set_font(font_name, size=12)
        pdf.set_text_color(0, 0, 0)
        pdf.cell(0, 10, txt=f"Запрос #{i}", ln=True)

        pdf.set_font(font_name, size=11)
        pdf.multi_cell(0, 10, f"📥 Ввод:\n{item['input']}\n")
        pdf.multi_cell(0, 10, f"📤 Ответ:\n{item['output']}\n")
        pdf.multi_cell(0, 10, f"📊 Метрики:\nСемантическое сходство: {item['similarity']:.4f}\nИндекс читаемости: {item['readability']:.2f}\n")

        pdf.ln(5)

    # Возвращаем PDF как байты
    return pdf.output(dest='S')

# ==== Промпты ====
SYSTEM_PROMPT = """
Отвечай в таком формате:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

CUSTOM_PROMPT = """
Ты - экспертный синонимайзер экономических и финансовых определений. Твоя цель - упростить определения, заменяя трудные понятия их более простыми синонимами. Требования:
1. Без искажений смысла
2. На уровне 11 класса школы

Пример 1: Финансовая аналитика — это процесс сбора, обработки, анализа и интерпретации многомерных финансовых и экономических данных с целью выявления закономерностей, оценки рисков, прогнозирования тенденций и поддержки принятия управленческих решений.
Ожидаемый ответ: Простыми словами, финансовая аналитика — действия людей, которые собирают информацию о деньгах, компаниях и экономике, анализируют её, чтобы понять, как всё это работает, и помогают руководителям принимать правильные решения.

Пример 2: Монетарная политика — это комплекс мер центрального банка, направленных на регулирование денежной массы, процентных ставок и инфляции через корректировку ключевой ставки.
Ожидаемый ответ: Простыми словами, монетарная политика - это различные действия ЦБ, которые предпринимаются для управления объемом денег в стране и ключевой ставкой.
"""

# ==== Загрузка моделей ====
pipe, tokenizer = load_model()
metric_model = load_metric_model()

# ==== Название ====
st.markdown("<h1 style='text-align: center; margin-top: -40px;'>🧠 Синонимайзер экономических терминов</h1>", unsafe_allow_html=True)
st.markdown("### Введите определение:")
user_input = st.text_area("", height=200)

# ==== Инициализация системного промпта и истории ====
if "system_prompt" not in st.session_state:
    st.session_state.system_prompt = CUSTOM_PROMPT

if "history" not in st.session_state:
    st.session_state.history = []

# ==== Кнопки сброса ====
col1, col2, col3 = st.columns(3)
with col1:
    if st.button("🔄 Сбросить к SYSTEM_PROMPT"):
        st.session_state.system_prompt = SYSTEM_PROMPT
        st.rerun()
with col2:
    if st.button("🎯 Сбросить к CUSTOM_PROMPT"):
        st.session_state.system_prompt = CUSTOM_PROMPT
        st.rerun()
with col3:
    if st.button("🗑 Очистить историю"):
        st.session_state.history.clear()
        st.rerun()

# ==== Блок редактирования промпта ====
with st.expander("⚙️ Настройка системного промпта", expanded=False):
    st.markdown("Можешь отредактировать инструкцию, по которой модель упрощает определения.")
    st.session_state.system_prompt = st.text_area(
        "Кастомный промпт:",
        value=st.session_state.system_prompt,
        height=300
    )

# ==== Обработка запроса ====
if st.button("🔁 Упростить") and user_input.strip():
    start = time.time()

    prompt = f"{st.session_state.system_prompt.strip()}\n\nОтработай по формату со следующим определением:\n\n{user_input}"
    messages = [{"role": "user", "content": prompt}]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    output = pipe(
        formatted_prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.35,
        top_k=45,
        top_p=0.98,
        eos_token_id=tokenizer.eos_token_id
    )

    result = output[0]['generated_text'][len(formatted_prompt):].strip()
    end = time.time()

    # Метрики
    emb_orig = metric_model.encode(user_input, convert_to_tensor=True)
    emb_gen = metric_model.encode(result, convert_to_tensor=True)
    similarity = util.cos_sim(emb_orig, emb_gen).item()
    readability = flesch_readability_russian(result)

    # Сохраняем в историю
    st.session_state.history.append({
        "input": user_input,
        "output": result,
        "similarity": similarity,
        "readability": readability
    })

    # Вывод
    st.write("### ✅ Упрощённое определение:")
    st.success(result)
    st.caption(f"⏱ Время генерации: {end - start:.2f} сек.")

    st.markdown("### 📊 Метрики качества:")
    st.metric("Семантическое сходство (SBERT)", f"{similarity:.4f}")
    st.metric("Индекс читаемости (Флеш, RU)", f"{readability:.2f}")

# ==== Отображение истории ====
if st.session_state.history:
    st.markdown("## 🕘 История запросов")
    for i, item in enumerate(reversed(st.session_state.history), 1):
        with st.expander(f"Запрос #{len(st.session_state.history) - i + 1}", expanded=False):
            st.markdown("**📥 Ввод:**")
            st.code(item["input"], language="text")

            st.markdown("**📤 Ответ:**")
            st.success(item["output"])

            st.markdown("**📊 Метрики:**")
            st.metric("Семантическое сходство (SBERT)", f"{item['similarity']:.4f}")
            st.metric("Индекс читаемости (Флеш, RU)", f"{item['readability']:.2f}")

# ==== Сохранение истории запросов в PDF ====
if st.session_state.history:
    pdf_bytes = generate_pdf(st.session_state.history)
    b64 = base64.b64encode(pdf_bytes).decode()
    href = f'<a href="data:application/pdf;base64,{b64}" download="history.pdf">📄 Скачать историю в PDF</a>'
    st.markdown(href, unsafe_allow_html=True)

Overwriting app.py


In [15]:
import subprocess
subprocess.Popen(["streamlit", "run", "app.py"])

print(f"✅ Streamlit доступен по ссылке: {public_url}")

✅ Streamlit доступен по ссылке: NgrokTunnel: "https://814c-34-75-171-47.ngrok-free.app" -> "http://localhost:8501"


# Запустите код ниже, что б корректно закончить работу веб-сервиса:

In [16]:
# import os

# # Завершаем процессы Streamlit и Ngrok
# os.system("pkill streamlit")
# os.system("pkill ngrok")

# print("✅ Все процессы Streamlit и Ngrok завершены.")

✅ Все процессы Streamlit и Ngrok завершены.
